# Database creation
This file is used to play with the data and once the objective is reached, functions will be written in .py files in order to be used by the model part of the project.

In [4]:
from astropy.io import fits
from tabulate import tabulate
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from dataset_creation_utils import *
# Retrieve the contrast curves from the database
path_db = 'C:/Users/ludin/Documents/Master Thesis/Dataset_creation/SPHERE_DC_DATA/'
foldername = '2MASS J01543773+0043005_DB_H23_2014-10-07_ird_specal_dc_cADI_softsorting_200967/'
filename = 'ird_specal_dc-IRD_SPECAL_CONTRAST_CURVE_TABLE-contrast_curve_tab.fits'

if not os.path.exists(path_db):
    print('ERROR! Folder {} does not exist.'.format(path_db)) # exit doesn't print anything in jupyter notebook

if not os.path.exists(os.path.join(path_db, foldername)):
    print('ERROR! Folder {} does not exist.'.format(foldername))

if not os.path.exists(os.path.join(path_db, foldername, filename)):
    print('ERROR! File {} does not exist.'.format(filename))

with fits.open(os.path.join(path_db, foldername, filename)) as hdul:
    data = hdul[1].data #numpy record
    print(data.dtype)
    print(hdul[1].header['DATE-OBS'])
    print(hdul[1].data['TARGET_NAME'])


(numpy.record, [('TARGET_NAME', 'S3'), ('LAM', 'S10'), ('PIXSCALE', 'S10'), ('NSIGMA', 'S3'), ('REPERTORY', 'S55'), ('SEPARATION', '>f4', (1148,)), ('NSIGMA_CONTRAST', '>f4', (1148,))])
2014-10-08T05:51:18.0343
['FS4' 'FS4' 'FS4']


In [2]:
df = get_df_with_headers(path_db, ['ESO OBS ID', 'DATE-OBS', 'OBJECT'])

In [3]:
# Print the types of the columns
print(df.dtypes)

ESO OBS ID          int64
DATE-OBS           object
OBJECT             object
SEPARATION         object
NSIGMA_CONTRAST    object
dtype: object


In [4]:
write_stats_in_file(df, path_db)

name_exists = False


In [79]:
def plot_contrast_curves(df, path):
    """
    Plot the contrast curves.
    """

    if not os.path.exists(path):
        exit('ERROR! Folder {} does not exist.'.format(path))

    if not path.endswith('/'):
        path += '/'

    if not os.path.exists(path + 'plots/'):
        os.makedirs(path + 'plots/')
        
    path += 'plots/'

    if not ('SEPARATION' in df.keys() and 'NSIGMA_CONTRAST' in df.keys() and 'OBJECT' in df.keys() and 'DATE-OBS' in df.keys(), 'ESO OBS ID' in df.keys()):
        exit('ERROR! The dataframe must contain the columns SEPARATION, NSIGMA_CONTRAST, OBJECT, DATE-OBS and ESO OBS ID.')
        
    # Dataframes are not meant to access elements by index.
    df_dict = df.to_dict()

    for i in range(len(df_dict['OBJECT'])):
        break
        # Log transform of the contrast and warning suppression
        with np.errstate(divide='ignore' , invalid='ignore'):
            contrast = np.log10(df_dict['NSIGMA_CONTRAST'][i][0]) # There are some negative and zero values in the contrast curves, maybe process them before plotting

        plt.figure()
        plt.plot(df_dict['SEPARATION'][i][0], contrast, label=df_dict['OBJECT'][i])
        plt.xlabel('Separation (arcsec)')
        plt.ylabel('Contrast (5-sigma)')
        plt.title('Contrast curve for {} on {}'.format(df_dict['OBJECT'][i], df_dict['DATE-OBS'][i]))
        filename = os.path.join(path, 'contrast_curve_{}.png'.format(df_dict['ESO OBS ID'][i]))
        # print('Saving figure {}...'.format(filename))
        plt.plot()
        plt.savefig(filename)
        plt.close()

In [78]:
plot_contrast_curves(df, 'C:/Users/ludin/Documents/Master Thesis/Dataset_creation')

C:\Users\ludin\AppData\Local\Temp/ipykernel_6648/2029011093.py:41: RuntimeWarning: divide by zero encountered in log10
  contrast_curves = np.array([np.log10(df_dict['NSIGMA_CONTRAST'][i][0]) for i in range(len(df_dict['OBJECT']))])
C:\Users\ludin\AppData\Local\Temp/ipykernel_6648/2029011093.py:41: RuntimeWarning: invalid value encountered in log10
  contrast_curves = np.array([np.log10(df_dict['NSIGMA_CONTRAST'][i][0]) for i in range(len(df_dict['OBJECT']))])
C:\Users\ludin\AppData\Local\Temp/ipykernel_6648/2029011093.py:41: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  contrast_curves = np.array([np.log10(df_dict['NSIGMA_CONTRAST'][i][0]) for i in range(len(df_dict['OBJECT']))])
